# LSTM on Pynq
This notebook covers how to use the FPGA-accelerated BiLSTM implementation on PYNQ to perform optical character recognition (OCR) on Fraktur text.
![alt text](Fraktur_images/LSTM_Topology.png "BiLSTM for OCR")

## 1. Instantiate the OCR model

Instantiating the OCR model will automatically download the appropriate bitstream onto the device and load the weights trained on the Fraktur dataset. 

In [1]:
import lstm

hw_ocr = lstm.PynqFrakturOCR(lstm.RUNTIME_HW)

## 2. Load a line of text
Load a line of Fraktur text that has been extracted from the Fraktur text "Wanderungen durch die Mark Brandenburg" by Theodor Fontane.

In [2]:
from PIL import Image
im = Image.open('/home/xilinx/jupyter_notebooks/lstm/Fraktur_images/010006.raw.lnrm.png').convert('L')
im

## 3. Perform OCR in hardware
Perform the OCR task on the input image using the hardware-accelerated OCR engine.

In [3]:
import numpy as np

hw_result = hw_ocr.inference(np.array(im))
hw_mops_per_s, hw_ms_inference_time, hw_recognized_text = hw_result

print("HW OCRed text: {}".format(hw_recognized_text))
print("HW MOps/s: {}".format(hw_mops_per_s))
print("HW inference time [ms]: {}".format(hw_ms_inference_time))

HW OCRed text: Für Andre hier gethan, sei stumme Gabe, -
HW MOps/s: 54833.779575112705
HW inference time [ms]: 1.7328369617462158


## 4. Perform OCR in software
For the sake of evaluating the performances of the hardware-accelerated OCR engine, perform the same OCR task using the Arm CPU.

In [4]:
sw_ocr = lstm.PynqFrakturOCR(lstm.RUNTIME_SW)

sw_result = sw_ocr.inference(np.array(im))
sw_mops_per_s, sw_ms_inference_time, sw_recognized_text = sw_result

print("SW OCRed text: {}".format(sw_recognized_text))
print("SW MOps/s: {}".format(sw_mops_per_s))
print("SW inference time [ms]: {}".format(sw_ms_inference_time))

SW OCRed text: Für Andre hier gethan, sei stumme Gabe, -
SW MOps/s: 1.2310819261405748
SW inference time [ms]: 77182.515625


## 5. Reset the hardware accelerator

In [5]:
hw_ocr.cleanup()